<a href="https://colab.research.google.com/github/VKSi/2020_12_DataCompetitions_Course/blob/main/HW_5_FeaturesEngineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Feature Engineering, Feature Selection, part I

Продолжим работу с данными, которые были использованы в ДЗ2 и 3, продолжим решать задачу обнаружения мошеннических транзакций, что позволит получить полное решение задачи / полный пайплайн.

### Задание 0:
 выбрать любую модель машнного обучения и зафиксировать любой тип валидации. Обучить базовую модель и зафиксировать базовое качество модели. В каждом следующем задании нужно будет обучить выбранную модель и оценивать ее качество на зафиксированной схеме валидации. После каждого задания, требуется сделать вывод о достигаемом качестве модели, по сравнению с качестом из предыдущего шага.

### Задание 1:
 признак TransactionDT - это смещение в секундах относительно базовой даты. Базовая дата - 2017-12-01, преобразовать признак TransactionDT в datetime, прибавив к базовой дате исходное значение признака. Из полученного признака выделить год, месяц, день недели, час, день.

### Задание 2:
 сделать конкатенацию признаков
* card1 + card2;
* card1 + card2 + card_3 + card_5;
* card1 + card2 + card_3 + card_5 + addr1 + addr2

Рассматривать их как категориальных признаки.

### Задание 3:
 Сделать FrequencyEncoder для признаков card1 - card6, addr1, addr2.

### Задание 4:
 Создать признаки на основе отношения: TransactionAmt к вычисленной статистике. Статистика - среднее значение / стандартное отклонение TransactionAmt, сгруппированное по card1 - card6, addr1, addr2, и по признакам, созданным в задании 2.

### Задание 5:
 Создать признаки на основе отношения: D15 к вычисленной статистике. Статистика - среднее значение / стандартное отклонение D15, сгруппированное по card1 - card6, addr1, addr2, и по признакам, созданным в задании 2.

### Задание 6:
 выделить дробную часть и целую часть признака TransactionAmt в два отдельных признака. После создать отдельных признак - логарифм от TransactionAmt

### Задание 7 (опция):
 выполнить предварительную подготовку / очистку признаков P_emaildomain и R_emaildomain (что и как делать - остается на ваше усмотрение) и сделать Frequency Encoding для очищенных признаков.

# Общая (подготовительная) часть

In [ ]:
!pip install catboost

     |████████████████████████████████| 66.3MB 53kB/s 


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from google.colab import files, drive
import pickle
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
from typing import List, Tuple
import catboost as cb

In [ ]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
PATH = "gdrive/My Drive/AI/GB/Kaggle/"
object_path = "gdrive/My Drive/AI/GB/Kaggle/objects/"

In [ ]:
def save_obj(obj, name: str):
    path = object_path + name + '.pkl'
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
    return path


def load_obj(name: str):
    with open(object_path + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
data = pd.read_csv(PATH + "assignment_2_train.csv")
lb = pd.read_csv(PATH + "assignment_2_test.csv")

In [ ]:
print("data.shape = {} rows, {} cols".format(*data.shape))
print("lb.shape = {} rows, {} cols".format(*lb.shape))

data.shape = 180000 rows, 394 cols
lb.shape = 100001 rows, 394 cols


In [ ]:
REST_FEATURES = data.columns
INITIAL_FEATURES = REST_FEATURES.copy()

ID = 'TransactionID'
REST_FEATURES = list(set(REST_FEATURES) - set([ID]))

TARGET = 'isFraud'
REST_FEATURES = list(set(REST_FEATURES) - set([TARGET]))

NUMERICAL_FEATURES = data[REST_FEATURES].select_dtypes(include=[np.number]).columns.to_list()
REST_FEATURES = list(set(REST_FEATURES) - set(NUMERICAL_FEATURES))

CATEGORICAL_FEATURES = REST_FEATURES

REST_FEATURES = None

In [ ]:
data[TARGET].value_counts()/data.shape[0]

0    0.971439
1    0.028561
Name: isFraud, dtype: float64

In [ ]:
data[TARGET].value_counts()/data.shape[0]

0    0.971439
1    0.028561
Name: isFraud, dtype: float64

In [ ]:
data[CATEGORICAL_FEATURES] = data[CATEGORICAL_FEATURES].astype('str')
lb[CATEGORICAL_FEATURES] = lb[CATEGORICAL_FEATURES].astype('str')

In [ ]:
X = data.drop([ID, TARGET], axis='columns')
y = data[TARGET]

In [ ]:
X_lb = lb.drop([ID, TARGET], axis='columns')
y_lb = lb[TARGET]

# Задание 0:
 выбрать любую модель машинного обучения и зафиксировать любой тип валидации. Обучить базовую модель и зафиксировать базовое качество модели. В каждом следующем задании нужно будет обучить выбранную модель и оценивать ее качество на зафиксированной схеме валидации. После каждого задания, требуется сделать вывод о достигаемом качестве модели, по сравнению с качестом из предыдущего шага.

Доработаем функцию из лекции, чтобы она работала с catboost

In [ ]:
def make_cross_validation(X: pd.DataFrame,
                          y: pd.Series,
                          estimator: object,
                          metric: callable,
                          cv_strategy):
    """
    Кросс-валидация.

    Parameters
    ----------
    X: pd.DataFrame
        Матрица признаков.

    y: pd.Series
        Вектор целевой переменной.

    estimator: callable
        Объект модели для обучения.

    metric: callable
        Метрика для оценки качества решения.
        Ожидается, что на вход будет передана функция,
        которая принимает 2 аргумента: y_true, y_pred.

    cv_strategy: cross-validation generator
        Объект для описания стратегии кросс-валидации.
        Ожидается, что на вход будет передан объект типа
        KFold или StratifiedKFold.

    Returns
    -------
    oof_score: float
        Значение метрики качества на OOF-прогнозах.

    fold_train_scores: List[float]
        Значение метрики качества на каждом обучающем датасете кросс-валидации.

    fold_valid_scores: List[float]
        Значение метрики качества на каждом валидационном датасете кросс-валидации.

    oof_predictions: np.array
        Прогнозы на OOF.

    """
    estimators, fold_train_scores, fold_valid_scores = [], [], []
    oof_predictions = np.zeros(X.shape[0])

    for fold_number, (train_idx, valid_idx) in enumerate(cv_strategy.split(X, y)):
        X_train, X_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]

        train_pool = cb.Pool(X_train, y_train, cat_features=CATEGORICAL_FEATURES)
        valid_pool = cb.Pool(X_valid, y_valid, cat_features=CATEGORICAL_FEATURES)
        estimator.fit(train_pool, eval_set=[train_pool, valid_pool])

        y_train_pred = estimator.predict_proba(X_train)[:, 1]
        y_valid_pred = estimator.predict_proba(X_valid)[:, 1]

        fold_train_scores.append(metric(y_train, y_train_pred))
        fold_valid_scores.append(metric(y_valid, y_valid_pred))
        oof_predictions[valid_idx] = y_valid_pred

        msg = (
            f"Fold: {fold_number+1}, train-observations = {len(train_idx)}, "
            f"valid-observations = {len(valid_idx)}\n"
            f"train-score = {round(fold_train_scores[fold_number], 4)}, "
            f"valid-score = {round(fold_valid_scores[fold_number], 4)}" 
        )
        print(msg)
        print("="*69)
        estimators.append(estimator)
        # save_obj(estimators, 'estimators_task_5')

    oof_score = metric(y, oof_predictions)
    print(f"CV-results train: {round(np.mean(fold_train_scores), 4)} +/- {round(np.std(fold_train_scores), 3)}")
    print(f"CV-results valid: {round(np.mean(fold_valid_scores), 4)} +/- {round(np.std(fold_valid_scores), 3)}")
    print(f"OOF-score = {round(oof_score, 4)}")

    return estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions

In [ ]:
cb_params = {
    "n_estimators": 10000,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": 500,
    "max_depth": 4,
    "reg_lambda": 100,
    "class_weights":[3, 97], 
    "early_stopping_rounds": 50,
    "thread_count": 6,
    "random_seed": 26
}

In [ ]:
model = cb.CatBoostClassifier(**cb_params)

In [ ]:
cv_strategy = StratifiedKFold(n_splits=5)

In [ ]:
estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = \
    make_cross_validation(X, y, model, metric=roc_auc_score, cv_strategy=cv_strategy)

0:	test: 0.7899082	test1: 0.7589028	best: 0.7589028 (0)	total: 706ms	remaining: 1h 57m 43s
500:	test: 0.9144923	test1: 0.8797447	best: 0.8797447 (500)	total: 5m 9s	remaining: 1h 37m 57s
1000:	test: 0.9402914	test1: 0.8967962	best: 0.8967962 (1000)	total: 10m 24s	remaining: 1h 33m 31s
1500:	test: 0.9533592	test1: 0.9023404	best: 0.9023750 (1495)	total: 15m 34s	remaining: 1h 28m 12s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.902803538
bestIteration = 1562

Shrink model to first 1563 iterations.
Fold: 1, train-observations = 144000, valid-observations = 36000
train-score = 0.9547, valid-score = 0.9028
0:	test: 0.7875437	test1: 0.8073380	best: 0.8073380 (0)	total: 681ms	remaining: 1h 53m 33s
500:	test: 0.9138769	test1: 0.9010069	best: 0.9010069 (500)	total: 5m 3s	remaining: 1h 35m 46s
1000:	test: 0.9401868	test1: 0.9115466	best: 0.9115488 (990)	total: 10m 6s	remaining: 1h 30m 51s
1500:	test: 0.9522268	test1: 0.9159292	best: 0.9159292 (1500)	total: 15m 9s	remaining:

In [ ]:
save_obj(estimators, 'estimators_HW5_task0')

'gdrive/My Drive/AI/GB/Kaggle/objects/estimators_HW5_task0.pkl'

In [ ]:
proba = np.zeros((len(estimators), X.shape[0]))
for i, est in enumerate(estimators):
  proba[i, :] = est.predict_proba(X)[:, 1]
proba_train = np.mean(proba, axis = 0)
train_score = roc_auc_score(y, proba_train)
print(f'ROC AUC train score = {round(train_score, 3)}')

ROC AUC train score = 0.94


In [ ]:
def my_validation(X, y, task_n):
  estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = \
    make_cross_validation(X, y, model, metric=roc_auc_score, cv_strategy=cv_strategy)
  save_obj(estimators, f'estimators_HW5_task{task_n}')
  proba = np.zeros((len(estimators), X.shape[0]))
  for i, est in enumerate(estimators):
    proba[i, :] = est.predict_proba(X)[:, 1]
  proba_train = np.mean(proba, axis = 0)
  train_score = roc_auc_score(y, proba_train)
  print(f'ROC AUC train score = {round(train_score, 3)}')    

# Задание 1:
Признак TransactionDT - это смещение в секундах относительно базовой даты. Базовая дата - 2017-12-01, преобразовать признак TransactionDT в datetime, прибавив к базовой дате исходное значение признака. Из полученного признака выделить год, месяц, день недели, час, день.

In [ ]:
data['TransactionDT'] = pd.to_datetime(data.TransactionDT, origin='2017-12-01', unit='s')
data.TransactionDT.head()

0   2017-12-02 00:00:00
1   2017-12-02 00:00:01
2   2017-12-02 00:01:09
3   2017-12-02 00:01:39
4   2017-12-02 00:01:46
Name: TransactionDT, dtype: datetime64[ns]

In [ ]:
data['TransactionDT_year'] = data.TransactionDT.dt.year
data['TransactionDT_month'] = data.TransactionDT.dt.month
data['TransactionDT_day'] = data.TransactionDT.dt.day
data['TransactionDT_hour'] = data.TransactionDT.dt.hour
data['TransactionDT_dow'] = data.TransactionDT.dt.dayofweek
cols = [col for col in data.columns if col[:13] == 'TransactionDT']
data[cols].head()

,TransactionDT,TransactionDT_year,TransactionDT_month,TransactionDT_day,TransactionDT_hour,TransactionDT_dow
0,2017-12-02 00:00:00,2017,12,2,0,5
1,2017-12-02 00:00:01,2017,12,2,0,5
2,2017-12-02 00:01:09,2017,12,2,0,5
3,2017-12-02 00:01:39,2017,12,2,0,5
4,2017-12-02 00:01:46,2017,12,2,0,5


In [ ]:
X = data.drop([ID, TARGET], axis='columns')
y = data[TARGET]

In [ ]:
my_validation(X, y, 1)

0:	test: 0.7742663	test1: 0.7632443	best: 0.7632443 (0)	total: 607ms	remaining: 1h 41m 10s
500:	test: 0.9144821	test1: 0.8788737	best: 0.8788737 (500)	total: 5m 11s	remaining: 1h 38m 33s
1000:	test: 0.9412517	test1: 0.8944355	best: 0.8944372 (998)	total: 10m 24s	remaining: 1h 33m 38s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.8955071784
bestIteration = 1060

Shrink model to first 1061 iterations.
Fold: 1, train-observations = 144000, valid-observations = 36000
train-score = 0.9428, valid-score = 0.8955
0:	test: 0.7688947	test1: 0.7959285	best: 0.7959285 (0)	total: 615ms	remaining: 1h 42m 28s
500:	test: 0.9137869	test1: 0.8990091	best: 0.8990091 (500)	total: 5m 10s	remaining: 1h 38m 15s
1000:	test: 0.9413172	test1: 0.9089941	best: 0.9089941 (1000)	total: 10m 20s	remaining: 1h 33m
1500:	test: 0.9533595	test1: 0.9130811	best: 0.9131177 (1493)	total: 15m 27s	remaining: 1h 27m 29s
2000:	test: 0.9614472	test1: 0.9166613	best: 0.9166613 (2000)	total: 20m 31s	remaining

### Результат:
CV-results train: 0.9038 +/- 0.046     
CV-results valid: 0.876 +/- 0.028     
OOF-score = 0.8527     
ROC AUC train score = 0.889     

При базовом:
CV-results train: 0.9192 +/- 0.043     
CV-results valid: 0.89 +/- 0.024     
OOF-score = 0.8757    
ROC AUC train score = 0.94

**Результат ухудшился**

# Задание 2:
сделать конкатенацию признаков

card1 + card2;
card1 + card2 + card_3 + card_5;
card1 + card2 + card_3 + card_5 + addr1 + addr2
Рассматривать их как категориальных признаки.

In [ ]:
mix_1 = {'card1', 'card2'}
mix_2 = mix_1.union({'card3', 'card5'})
mix_3 = mix_2.union({'addr1', 'addr2'})

for feat in mix_3:
  print('*'*20)
  print(feat)
  print(data[feat].nunique())

********************
addr2
54
********************
addr1
269
********************
card1
9496
********************
card5
95
********************
card3
88
********************
card2
499


In [ ]:
TO_CATEGORICAL = []
for mix in [mix_1, mix_2, mix_3]:
  feat_name = '_'.join(mix)
  TO_CATEGORICAL.append(feat_name)
  data[feat_name] = ''
  for feat in mix:
      data[feat_name] = data[feat_name] + data[feat].astype('str') +' | '
  data[feat_name] = data[feat_name].str[:-3]

In [ ]:
TO_CATEGORICAL

['card1_card2',
 'card3_card5_card1_card2',
 'addr2_addr1_card1_card5_card3_card2']

In [ ]:
CATEGORICAL_FEATURES += TO_CATEGORICAL
CATEGORICAL_FEATURES

['M4',
 'M5',
 'M8',
 'card6',
 'card4',
 'R_emaildomain',
 'M7',
 'M6',
 'M9',
 'M3',
 'M1',
 'P_emaildomain',
 'ProductCD',
 'M2',
 'card1_card2',
 'card3_card5_card1_card2',
 'addr2_addr1_card1_card5_card3_card2']

In [ ]:
X = data.drop([ID, TARGET], axis='columns')
y = data[TARGET]

In [ ]:
my_validation(X, y, 2)

0:	test: 0.8763395	test1: 0.7543158	best: 0.7543158 (0)	total: 903ms	remaining: 2h 30m 30s
500:	test: 0.9804959	test1: 0.9105092	best: 0.9105228 (499)	total: 6m 5s	remaining: 1h 55m 34s
1000:	test: 0.9874537	test1: 0.9197185	best: 0.9197185 (1000)	total: 12m 4s	remaining: 1h 48m 36s
1500:	test: 0.9901774	test1: 0.9220240	best: 0.9220352 (1499)	total: 18m 1s	remaining: 1h 42m 3s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9221167095
bestIteration = 1514

Shrink model to first 1515 iterations.
Fold: 1, train-observations = 144000, valid-observations = 36000
train-score = 0.9902, valid-score = 0.9221
0:	test: 0.7749248	test1: 0.8013034	best: 0.8013034 (0)	total: 792ms	remaining: 2h 11m 57s
500:	test: 0.9820669	test1: 0.9252751	best: 0.9252751 (500)	total: 6m	remaining: 1h 53m 53s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9308730753
bestIteration = 724

Shrink model to first 725 iterations.
Fold: 2, train-observations = 144000, valid-observ

### Результат пока самый лучший!

# Задание 3:
Сделать FrequencyEncoder для признаков card1 - card6, addr1, addr2.

In [ ]:
TO_HANDLE = mix_3.union({'card4', 'card6'})
TO_HANDLE

{'addr1', 'addr2', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6'}

In [ ]:
for feat in TO_HANDLE:
  data[f'{feat}_FRQ'] = data[feat].map(data[feat].value_counts())

In [ ]:
X = data.drop([ID, TARGET], axis='columns')
y = data[TARGET]

In [ ]:
my_validation(X, y, 3)

0:	test: 0.8338584	test1: 0.7741458	best: 0.7741458 (0)	total: 902ms	remaining: 2h 30m 22s
500:	test: 0.9800831	test1: 0.9128857	best: 0.9128857 (500)	total: 5m 45s	remaining: 1h 49m 2s
1000:	test: 0.9880490	test1: 0.9236569	best: 0.9237095 (972)	total: 11m 33s	remaining: 1h 43m 58s
1500:	test: 0.9907329	test1: 0.9266562	best: 0.9266606 (1495)	total: 17m 16s	remaining: 1h 37m 51s
2000:	test: 0.9923155	test1: 0.9279831	best: 0.9280199 (1966)	total: 22m 52s	remaining: 1h 31m 27s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9288524761
bestIteration = 2259

Shrink model to first 2260 iterations.
Fold: 1, train-observations = 144000, valid-observations = 36000
train-score = 0.9929, valid-score = 0.9289
0:	test: 0.8284719	test1: 0.7520158	best: 0.7520158 (0)	total: 1.43s	remaining: 3h 58m 23s
500:	test: 0.9801566	test1: 0.9265693	best: 0.9265693 (500)	total: 5m 43s	remaining: 1h 48m 40s
1000:	test: 0.9881141	test1: 0.9346528	best: 0.9346786 (990)	total: 11m 26s	remaini

### Результат еще улучшился
CV-results train: 0.9649 +/- 0.027     
CV-results valid: 0.8997 +/- 0.031     
OOF-score = 0.8859     
ROC AUC train score = 0.967     

# Задание 4:
Создать признаки на основе отношения: TransactionAmt к вычисленной статистике. Статистика - среднее значение / стандартное отклонение TransactionAmt, сгруппированное по card1 - card6, addr1, addr2, и по признакам, созданным в задании 2

In [ ]:
TO_HANDLE = TO_HANDLE.union(set(TO_CATEGORICAL))
TO_HANDLE

{'addr1',
 'addr2',
 'addr2_addr1_card1_card5_card3_card2',
 'card1',
 'card1_card2',
 'card2',
 'card3',
 'card3_card5_card1_card2',
 'card4',
 'card5',
 'card6'}

In [ ]:
AGGREGATED_FEAT = 'TransactionAmt'

In [ ]:
def z_score(df_, AGGREGATED_FEAT_, feat_):
  df_tt = df_[[AGGREGATED_FEAT_, feat_]].copy()
  m = df_tt[[feat_, AGGREGATED_FEAT_]].groupby(feat_).mean().squeeze()
  s = df_tt[[feat_, AGGREGATED_FEAT_]].groupby(feat_).std().squeeze()
  df_tt['z_score'] = (df_tt[AGGREGATED_FEAT] - df_tt[feat_].map(m))/df_tt[feat_].map(s)
  return df_tt['z_score']

In [ ]:
stats_name = 'z_score'
for feat in TO_HANDLE:
  data[f'{AGGREGATED_FEAT}__{stats_name.upper()}_BY__{feat}'] = z_score(data, AGGREGATED_FEAT, feat)

In [ ]:
X = data.drop([ID, TARGET], axis='columns')
y = data[TARGET]

In [ ]:
my_validation(X, y, 4)

0:	test: 0.8190092	test1: 0.7778811	best: 0.7778811 (0)	total: 772ms	remaining: 2h 8m 36s
500:	test: 0.9803433	test1: 0.9118932	best: 0.9118932 (500)	total: 5m 49s	remaining: 1h 50m 34s
1000:	test: 0.9881566	test1: 0.9205431	best: 0.9205431 (1000)	total: 11m 35s	remaining: 1h 44m 16s
1500:	test: 0.9907188	test1: 0.9230281	best: 0.9230340 (1499)	total: 17m 20s	remaining: 1h 38m 11s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9251364683
bestIteration = 1881

Shrink model to first 1882 iterations.
Fold: 1, train-observations = 144000, valid-observations = 36000
train-score = 0.9921, valid-score = 0.9251
0:	test: 0.8752692	test1: 0.7950245	best: 0.7950245 (0)	total: 748ms	remaining: 2h 4m 41s
500:	test: 0.9792226	test1: 0.9254715	best: 0.9254715 (500)	total: 5m 45s	remaining: 1h 49m 11s
1000:	test: 0.9887211	test1: 0.9336643	best: 0.9336771 (995)	total: 11m 33s	remaining: 1h 43m 58s
1500:	test: 0.9911265	test1: 0.9360165	best: 0.9360262 (1494)	total: 17m 5s	remainin

### Результат улучшился
CV-results train: 0.9637 +/- 0.031     
CV-results valid: 0.9116 +/- 0.017     
OOF-score = 0.8905     
ROC AUC train score = 0.968

### Задание 5:
Создать признаки на основе отношения: D15 к вычисленной статистике. Статистика - среднее значение / стандартное отклонение D15, сгруппированное по card1 - card6, addr1, addr2, и по признакам, созданным в задании 2.

In [ ]:
AGGREGATED_FEAT = 'D15'

In [ ]:
stats = {'mean': np.mean, 'std': np.std}
for feat in TO_HANDLE:
  for stat_name, stat_func in stats.items():
    to_map = data[[feat, AGGREGATED_FEAT]].groupby(feat).agg(stat_func).squeeze()
    data[f'{AGGREGATED_FEAT}__{stat_name.upper()}_RATIO__{feat}'] = data[AGGREGATED_FEAT]/data[feat].map(to_map)

In [ ]:
X = data.drop([ID, TARGET], axis='columns')
y = data[TARGET]

In [ ]:
my_validation(X, y, 5)

0:	test: 0.7716409	test1: 0.7345948	best: 0.7345948 (0)	total: 802ms	remaining: 2h 13m 40s
500:	test: 0.9807196	test1: 0.9118810	best: 0.9118810 (500)	total: 5m 57s	remaining: 1h 52m 55s
1000:	test: 0.9879156	test1: 0.9217803	best: 0.9218304 (992)	total: 11m 53s	remaining: 1h 46m 55s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9222425467
bestIteration = 1065

Shrink model to first 1066 iterations.
Fold: 1, train-observations = 144000, valid-observations = 36000
train-score = 0.9881, valid-score = 0.9222
0:	test: 0.8316072	test1: 0.8015972	best: 0.8015972 (0)	total: 711ms	remaining: 1h 58m 31s
500:	test: 0.9818739	test1: 0.9261738	best: 0.9261738 (500)	total: 5m 50s	remaining: 1h 50m 51s
1000:	test: 0.9889747	test1: 0.9335081	best: 0.9335496 (976)	total: 11m 41s	remaining: 1h 45m 3s
1500:	test: 0.9912887	test1: 0.9354192	best: 0.9354216 (1497)	total: 17m 21s	remaining: 1h 38m 15s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9361592943
bestI

### Результат голосования моделей из фолдов на всех данных несколько вырос, однако средний результат по моделям не улучшился (OOF-score даже упал)
CV-results train: 0.964 +/- 0.033     
CV-results valid: 0.9001 +/- 0.031     
OOF-score = 0.8766     
ROC AUC train score = 0.975     

# Задание 6:
выделить дробную часть и целую часть признака TransactionAmt в два отдельных признака. После создать отдельных признак - логарифм от TransactionAmt

In [ ]:
AGGREGATED_FEAT = 'TransactionAmt'

In [ ]:
data[f'{AGGREGATED_FEAT}__ROUNDED'] = data[AGGREGATED_FEAT].apply(round)
data[f'{AGGREGATED_FEAT}__FRACTIONAL'] = data[AGGREGATED_FEAT] - data[AGGREGATED_FEAT].apply(round)
data[f'{AGGREGATED_FEAT}__LOG'] = data[AGGREGATED_FEAT].apply(np.log)

In [ ]:
X = data.drop([ID, TARGET], axis='columns')
y = data[TARGET]

In [ ]:
my_validation(X, y, 6)

0:	test: 0.7713164	test1: 0.7437848	best: 0.7437848 (0)	total: 789ms	remaining: 2h 11m 33s
500:	test: 0.9769060	test1: 0.9107825	best: 0.9107825 (500)	total: 5m 58s	remaining: 1h 53m 18s
1000:	test: 0.9871105	test1: 0.9206851	best: 0.9206988 (999)	total: 11m 55s	remaining: 1h 47m 9s
1500:	test: 0.9901635	test1: 0.9239470	best: 0.9240014 (1488)	total: 17m 44s	remaining: 1h 40m 28s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9244514288
bestIteration = 1688

Shrink model to first 1689 iterations.
Fold: 1, train-observations = 144000, valid-observations = 36000
train-score = 0.9909, valid-score = 0.9245
0:	test: 0.7982959	test1: 0.7519856	best: 0.7519856 (0)	total: 723ms	remaining: 2h 24s
500:	test: 0.9812255	test1: 0.9264107	best: 0.9264107 (500)	total: 5m 51s	remaining: 1h 50m 55s
1000:	test: 0.9888012	test1: 0.9336308	best: 0.9336379 (999)	total: 11m 43s	remaining: 1h 45m 27s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9345039122
bestItera

### Результат ухудшился относительно максимального
CV-results train: 0.9621 +/- 0.028     
CV-results valid: 0.8978 +/- 0.031     
OOF-score = 0.8858     
ROC AUC train score = 0.958